# Loading_Data + Train/Dev Split

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from google.colab import drive
drive.mount('/content/drive')
from PIL import Image
from skimage import io


from sklearn.metrics import confusion_matrix

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Loading data function
def image_loader(folder_path,X,y,label):
  for filename in os.listdir(folder_path):
      if filename.endswith(".jpeg") or filename.endswith(".png"):
          img = io.imread(os.path.join(folder_path, filename), as_gray=False)
          X.append(img)
          y.append(label)

In [ ]:
y = []
X = []
##Load Hurricane Damaged Image
folder_path = "/content/drive/MyDrive/Hurricane_Damage_Satellite/train_dev/damage"
label = 1
image_loader(folder_path, X, y, label)

In [ ]:
##Load Hurricane unDamaged Image
folder_path = "/content/drive/MyDrive/Hurricane_Damage_Satellite/train_dev/no_damage"
label = 0
image_loader(folder_path, X, y, label)

In [ ]:
#Make X and y np array
X = np.array(X)
y = np.array(y)

#Train/Dev Split stratified on class labels
from sklearn.model_selection import train_test_split
X_tr, X_dev, y_tr, y_dev = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

#Check shape
X_tr.shape

(18984, 128, 128, 3)

In [ ]:
#Load Test Set dmage
y_te = []
X_te = []
label = 1
folder_path = "/content/drive/MyDrive/Hurricane_Damage_Satellite/test/damage"
image_loader(folder_path, X_te, y_te, label)

In [ ]:
#Load Test Set no_dmage
label = 0
folder_path = "/content/drive/MyDrive/Hurricane_Damage_Satellite/test/no_damage"
image_loader(folder_path, X_te, y_te, label)

X_te = np.array(X_te)
y_te = np.array(y_te)

In [ ]:
y_te.shape

(2000,)

# RF on flattened image

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF_clf = RandomForestClassifier()

In [ ]:
X_tr_flattened = X_tr.reshape(X_tr.shape[0], -1)
X_dev_flattened = X_dev.reshape(X_dev.shape[0], -1)
X_te_flattened = X_te.reshape(X_te.shape[0], -1)

In [ ]:
X_dev_flattened.shape

(2016, 49152)

In [ ]:
RF_clf.fit(X_tr_flattened, y_tr)

RandomForestClassifier()

In [ ]:
RF_pred_tr = RF_clf.predict(X_tr_flattened)
RF_accuracy_tr = np.mean(RF_pred_tr == y_tr)
print("Tranning accuracy = ", RF_accuracy_tr)

Tranning accuracy =  1.0


In [ ]:
RF_pred_dev = RF_clf.predict(X_dev_flattened)
RF_accuracy_dev = np.mean(RF_pred_dev == y_dev)
print("Validation accuracy = ", RF_accuracy_dev)

Validation accuracy =  0.8546626984126984


In [ ]:
RF_pred_te = RF_clf.predict(X_te_flattened)
RF_accuracy_te = np.mean(RF_pred_te == y_te)
print("Test accuracy = ", RF_accuracy_te)

Test accuracy =  0.967


In [ ]:
cm_dev = confusion_matrix(y_dev, RF_pred_dev)
print(cm_dev)

[[ 464  208]
 [  85 1259]]


In [ ]:
cm_te = confusion_matrix(y_te, RF_pred_te)
print(cm_te)

[[948  52]
 [ 14 986]]


# CNN Baseline

In [ ]:
from __future__ import print_function
import tensorflow as tf 
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define the input shape
input_shape = (128, 128, 3)

# Create a Sequential model
model = Sequential()

# Block 1
model.add(Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu', padding='valid', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

# Block 2
model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

# Block 3
model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

# Dense layers
model.add(Flatten())
model.add(Dense(units=4096, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(units=4096, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(units = 2, activation='sigmoid'))

# Print the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 30, 30, 96)        34944     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 96)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 14, 14, 256)       614656    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 6, 6, 256)        0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 6, 6, 384)         885120    
                                                                 
 conv2d_8 (Conv2D)           (None, 6, 6, 384)        

In [ ]:
# Convert class vectors to 10 class matrices
y_train = keras.utils.np_utils.to_categorical(y_tr, 2)
y_val = keras.utils.np_utils.to_categorical(y_dev, 2)
y_test = keras.utils.np_utils.to_categorical(y_te, 2)

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(learning_rate = 0.5),
              metrics=['accuracy'])

# Train the model using full batch
model.fit(X_tr, y_train,
          epochs = 30,
          verbose=1,
          validation_data=(X_dev, y_val))

Epoch 1/30
594/594 [==============================] - 6s 6ms/step - loss: 0.8038 - accuracy: 0.6843 - val_loss: 0.4171 - val_accuracy: 0.8219
Epoch 2/30
594/594 [==============================] - 3s 6ms/step - loss: 0.4262 - accuracy: 0.8138 - val_loss: 0.3874 - val_accuracy: 0.8279
Epoch 3/30
594/594 [==============================] - 3s 6ms/step - loss: 0.3758 - accuracy: 0.8441 - val_loss: 0.3316 - val_accuracy: 0.8740
Epoch 4/30
594/594 [==============================] - 3s 6ms/step - loss: 0.3542 - accuracy: 0.8632 - val_loss: 0.6228 - val_accuracy: 0.6404
Epoch 5/30
594/594 [==============================] - 3s 6ms/step - loss: 0.3032 - accuracy: 0.8813 - val_loss: 0.2710 - val_accuracy: 0.8924
Epoch 6/30
594/594 [==============================] - 3s 6ms/step - loss: 0.2768 - accuracy: 0.8954 - val_loss: 0.3641 - val_accuracy: 0.8472
Epoch 7/30
594/594 [==============================] - 3s 6ms/step - loss: 0.2459 - accuracy: 0.9094 - val_loss: 0.2638 - val_accuracy: 0.8993
Epoch 

In [ ]:
# Test accuracy after 30 epochs
score = model.evaluate(X_te, y_test, verbose=0)
print('Test accuracy:', score[1])

Test accuracy: 0.9704999923706055


In [ ]:
# Dev Set CM
pred_dev_CNN = np.argmax(model.predict(X_dev),axis=1)
cm_CNN = confusion_matrix(pred_dev_CNN, y_dev)
print(cm_CNN)

63/63 [==============================] - 0s 2ms/step
[[ 536   75]
 [ 136 1269]]


In [ ]:
# Test Set CM
pred_te_CNN = np.argmax(model.predict(X_te),axis=1)
cm_CNN = confusion_matrix(pred_te_CNN, y_te)
print(cm_CNN)

63/63 [==============================] - 0s 2ms/step
[[949   8]
 [ 51 992]]
